## Overview

Repo: https://github.com/pgahq/instructor-groq-openai-llm-examples

This notebook shows how to use Instructor with LLMs to keep things clean and avoid having to write/generate the messy function schema.

The magic comes from `OpenAISchema` in Instructor. Each tool is defined like this:

```
class MyTool(OpenAISchema):
    """
    Detailed description here tells the LLM when and how to use this tool.
    """
    param1: str = Field(..., description="describe this param")
    param2: int = Field(..., description="describe this param")

    def run(self):
      # the actual tool code goes here
      return(results from the tool)
```

Note: this notebook assumes you're using Google Colab. You can safely edit / play here. Or go to `File` -> `Save a copy in Google Drive` to make your own version.

In [1]:
%%capture
!pip install instructor groq openai

On the left, click the key and set two secrets with your keys. Be sure to enable "Notebook access" for them. This is how Google Colab works...you're not sharing your keys with anyone.

OPENAI_API_KEY

GROQ_API_KEY

In [2]:
from instructor import OpenAISchema   # this is what makes everything possible
import openai
import groq
from pydantic import BaseModel, Field, field_validator
from typing import List
import subprocess
from rich import print as rprint
import os

try:
    from google.colab import userdata
    os.environ['OPENAI_API_KEY'] = '' or userdata.get('OPENAI_API_KEY') # or put your key in the '' on this line
    os.environ['GROQ_API_KEY'] = '' or userdata.get('GROQ_API_KEY')
except Exception as e:
    # print(e)
    pass


A couple boring util functions that will run the tools we'll define later.

In [4]:
def get_completion_and_use_tools(messages, tool_functions, client, model):
    while True:   # loop until the requested tool(s) has been used and/or a normal text response has been provided by the LLM
        completion = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=[{"type": "function", "function": func.openai_schema} for func in tool_functions],   # func.openai_schema is the hero here
            tool_choice="auto",
            temperature=0.5,
            max_tokens=4096,
        )

        completion_message = completion.choices[0].message
        if completion_message.tool_calls is None:
            return(completion_message.content)  # no tool...just a regular LLM response
        else:
            messages.append(completion_message)   # chat completions requires this
            for tool_call in completion_message.tool_calls:
                print(f"\033[90m🛠️ {tool_call.function.name} {tool_call.function.arguments}\033[0m")
                tool_result = execute_tool(tool_call, tool_functions)   # run the requested tool
                messages.append({   # chat completions requires this
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": tool_call.function.name,
                    "content": tool_result,
                    })

def execute_tool(tool_call, funcs):
    # inspired by https://github.com/VRSEN/agency-swarm/threads/thread.py - original source?
    func = next(iter([func for func in funcs if func.__name__ == tool_call.function.name]))

    if not func:
        return f"Error: Function {tool_call.function.name} not found. Available functions: {[func.__name__ for func in funcs]}"
    try:
        func = func(**eval(tool_call.function.arguments))
        output = func.run()  # always execute only the run() function from the tool class
        return output
    except Exception as e:
        return "Error: " + str(e)

## Create the tools
With Instructor, tools are classes derived from `OpenAISchema`. When a tool gets called, its `run()` function will be executed (because that's how `execute_tool()` is defined above).

In [6]:
class GetWeather(OpenAISchema):
    """
    Determine weather in a location

    ## Limitations
    - Only returns temperature in fahrenheit
    """
    location: str = Field(..., description="The city and state e.g. San Francisco, CA")

    def run(self):      # implement the tool code here
      # this example tool implementation is hard coded with no actual logic
      return(f"Rainy, 48 degrees fahrenheit in {self.location}")

class GetDirections(OpenAISchema):
    """
    Get driving directions from one city to another.

    ## Limitations
    - Distances are only given in miles
    """
    from_city: str = Field(..., description="The name of the departure city")
    to_city: str = Field(..., description="The name of the destination city")

    def run(self):      # implement the tool code here
      # this example tool implementation is hard coded with no actual logic
      return(f"Drive north to get from {self.from_city} to {self.to_city}")

# collect the tool classes we just defined into an array so they can be used as tools later
tool_functions = [GetWeather, GetDirections]

## Let it fly
Pass in the prompt and the tools.

In [7]:
messages=[
    {"role": "system", "content": "You answer questions - short and sweet."},
    {"role": "user", "content": "What's the weather in NYC and how do I get there from Florida?"},    # the main prompt
]

print("gpt-4o via OpenAI:")
result = get_completion_and_use_tools(
            messages.copy(), # or pass messages directly to retain the tool calling chatter in the thread
            tool_functions,  # the array containing the OpenAISchema tool classes defined earlier
            openai.OpenAI(), # standard stuff
            "gpt-4o"    # model name
            )
print(result + "\n")

print("llama3-70b-8192 via Groq:")
print(get_completion_and_use_tools(messages.copy(), tool_functions, groq.Groq(), "llama3-70b-8192") + "\n")



gpt-4-turbo via OpenAI:
🛠️ GetWeather {"location": "New York, NY"}
🛠️ GetDirections {"from_city": "Florida", "to_city": "New York, NY"}
The weather in New York, NY is rainy with a temperature of 48°F. To get there from Florida, drive north.

llama3-70b-8192 via Groq:
🛠️ GetWeather {"location":"New York City, NY"}
🛠️ GetDirections {"from_city":"Florida","to_city":"New York City"}
Since you're planning to drive from Florida to New York City, I'll provide you with a more detailed response. Here's a concise answer: "To get to New York City from Florida, drive north. The current weather in NYC is rainy with a temperature of 48 degrees Fahrenheit."

